In [1]:
import os
import json
import re
import unicodedata
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from transformers import *
from tokenizers import *
from datasets import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
import nltk
from nltk.data import load
import pickle

/home/hdh/PycharmProjects/huggingface_bert/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# model_version = 'bert-base-uncased'
# model = BertModel.from_pretrained(model_version, output_attentions=True)

from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

https://huggingface.co/facebook/bart-large-mnli/resolve/main/config.json not found in cache or force_download set to True, downloading to /home/hdh/.cache/huggingface/transformers/tmpz3_4t30a
Downloading: 100%|██████████| 1.13k/1.13k [00:00<00:00, 591kB/s]
storing https://huggingface.co/facebook/bart-large-mnli/resolve/main/config.json in cache at /home/hdh/.cache/huggingface/transformers/980f2be6bd282c5079e99199d7554cfd13000433ed0fdc527e7def799e5738fe.4fdc7ce6768977d347b32986aff152e26fcebbda34ef89ac9b114971d0342e09
creating metadata file for /home/hdh/.cache/huggingface/transformers/980f2be6bd282c5079e99199d7554cfd13000433ed0fdc527e7def799e5738fe.4fdc7ce6768977d347b32986aff152e26fcebbda34ef89ac9b114971d0342e09
loading configuration file https://huggingface.co/facebook/bart-large-mnli/resolve/main/config.json from cache at /home/hdh/.cache/huggingface/transformers/980f2be6bd282c5079e99199d7554cfd13000433ed0fdc527e7def799e5738fe.4fdc7ce6768977d347b32986aff152e26fcebbda34ef89ac9b114971d0

Downloading: 100%|██████████| 446k/446k [00:00<00:00, 470kB/s]  
storing https://huggingface.co/facebook/bart-large-mnli/resolve/main/merges.txt in cache at /home/hdh/.cache/huggingface/transformers/19c09c9654551e163f858f3c99c226a8d0026acc4935528df3b09179204efe4c.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
creating metadata file for /home/hdh/.cache/huggingface/transformers/19c09c9654551e163f858f3c99c226a8d0026acc4935528df3b09179204efe4c.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
https://huggingface.co/facebook/bart-large-mnli/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /home/hdh/.cache/huggingface/transformers/tmpl7ovcvf5
Downloading: 100%|██████████| 1.29M/1.29M [00:00<00:00, 1.39MB/s]
storing https://huggingface.co/facebook/bart-large-mnli/resolve/main/tokenizer.json in cache at /home/hdh/.cache/huggingface/transformers/540455855ce0a3c13893c5d090d142de9481365bd32dc5457c957e5d13444d23.fc957603

* https://huggingface.co/models?pipeline_tag=zero-shot-classification&sort=downloads
* bert-base-uncased Fill-Mask Updated 4 days ago 33.1M 295
* xlm-roberta-base Fill-Mask Updated Jun 6 26.3M 65
* Jean-Baptiste/camembert-ner Token Classification Updated about 5 hours ago 17.1M 25
* openai/clip-vit-large-patch14 Zero-Shot Image Classification Updated about 11 hours ago 12.5M 26
* roberta-base Fill-Mask Updated about 10 hours ago 12M 55
* gpt2 Text Generation Updated about 10 hours ago 10.4M 231
* roberta-large Fill-Mask Updated about 10 hours ago 9.91M 49
* distilbert-base-uncased Fill-Mask Updated Jun 1 9.34M 82
* bert-base-cased Fill-Mask Updated Sep 6, 2021 6.53M 43
* microsoft/deberta-base Fill-Mask Updated 4 days ago 5.83M 30
* google/vit-base-patch16-224 Image Classification Updated Jun 23 5.06M 82
* bert-base-chinese Fill-Mask Updated Jul 22 4.04M 128
* sentence-transformers/all-MiniLM-L6-v2 Sentence Similarity Updated Jul 12 3.9M 88
* SpanBERT/spanbert-large-cased Updated May 19, 2021 3.6M 7
* openai/clip-vit-base-patch32 Zero-Shot Image Classification Updated 20 days ago 3.11M 63
* distilbert-base-uncased-finetuned-sst-2-english Text Classification Updated Aug 16 2.74M 95
* albert-base-v2 Fill-Mask Updated Aug 30, 2021 2.6M 18
* deepset/roberta-base-squad2 Question Answering Updated 9 days ago 2.56M 118
* unitary/toxic-bert Text Classification Updated Jun 8, 2021 2.49M 22
* facebook/bart-base Feature Extraction Updated Jun 3 2.29M 23
* sentence-transformers/paraphrase-MiniLM-L6-v2 Sentence Similarity Updated Jun 16 2.28M 22
* CompVis/stable-diffusion-safety-checker Updated Aug 22 2.05M 14
* distilgpt2 Text Generation Updated 24 days ago 1.83M 84
* bert-base-multilingual-cased Fill-Mask Updated Aug 8 1.83M 49
* ProsusAI/finbert Text Classification Updated Jun 3 1.8M 101
* t5-small Translation Updated Jul 22 1.78M 33
* facebook/contriever-msmarco Feature Extraction Updated Jun 26 1.63M
* csarron/bert-base-uncased-squad-v1 Question Answering Updated May 19, 2021 1.38M 1
* DeepPavlov/rubert-base-cased-conversational Feature Extraction Updated Nov 8, 2021 1.33M 6
* tals/albert-xlarge-vitaminc-mnli Text Classification Updated Aug 5 1.29M 1

In [ ]:
dataset_train = nlp.data.TSVDataset(".cache/train_H04W4_220511.txt", field_indices=[0,1], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset(".cache/test_H04W4_220511.txt", field_indices=[0,1], num_discard_samples=1)
# 텍스트 내에 줄바꿈과 탭이 존재하여 오류 발생, 처리 필요

In [ ]:
train_data_df = pd.read_csv('data/train_C09K11_220715.txt', sep='\t')  # text와 라벨 파일
test_dataset_df = pd.read_csv('data/test_C09K11_220715.txt', sep='\t')
train_dataset = Dataset.from_pandas(train_data_df)  # Dataset 객체 생성
test_dataset = Dataset.from_pandas(test_dataset_df)
finetune_dataset = DatasetDict()  # DatasetDict 객체 생성
finetune_dataset['train'] = train_dataset
finetune_dataset['test'] = test_dataset
finetune_dataset